# Example notebook for the PSPLIB Python package

This notebook demonstrates how to use the PSPLIB Python package for parsing different project scheduling formats, highlighting the differences between the resulting output.
All parsers return an instance of the [`ProjectInstance`](https://github.com/PyJobShop/PSPLIB/blob/main/psplib/ProjectInstance.py) class. 


In [1]:
from psplib import parse

## PSPLIB format

PSPLIB is a standard format for resource-constrained project scheduling problems (RCPSPs).

In [2]:
instance = parse("data/j301_1.sm", instance_format="psplib")

Get basic instance information: number of resources and activities

In [3]:
(instance.num_resources, instance.num_activities)

(4, 32)

Resource have a capacity and renewable status.

In [4]:
instance.resources

[Resource(capacity=12, renewable=True, skills=None),
 Resource(capacity=13, renewable=True, skills=None),
 Resource(capacity=4, renewable=True, skills=None),
 Resource(capacity=12, renewable=True, skills=None)]

Activities have modes, successors and possible delays (for RCPSP/max).

In [5]:
instance.activities[:4]

[Activity(modes=[Mode(duration=0, demands=[0, 0, 0, 0], skill_requirements=None)], successors=[1, 2, 3], delays=None, optional=False, selection_groups=[], name=''),
 Activity(modes=[Mode(duration=8, demands=[4, 0, 0, 0], skill_requirements=None)], successors=[5, 10, 14], delays=None, optional=False, selection_groups=[], name=''),
 Activity(modes=[Mode(duration=4, demands=[10, 0, 0, 0], skill_requirements=None)], successors=[6, 7, 12], delays=None, optional=False, selection_groups=[], name=''),
 Activity(modes=[Mode(duration=6, demands=[0, 0, 0, 3], skill_requirements=None)], successors=[4, 8, 9], delays=None, optional=False, selection_groups=[], name='')]

A mode is a single processing option for a given activity, which includes duration and demand per resource.

In [6]:
instance.activities[0].modes

[Mode(duration=0, demands=[0, 0, 0, 0], skill_requirements=None)]

## Patterson format

The Patterson format is another format for RCPSP instances.
The concent of the instance is identical to the ones from PSPLIB.

In [7]:
instance = parse("data/RG300_1.rcp", instance_format="patterson")

## RCPSP/max format

RCPSP/max instances introduce delays for each successor.

In [8]:
instance = parse("data/UBO10_01.sch", instance_format="rcpsp_max")

Note that the additional `delays` field is not empty, which specifies the time lag for each successor.

In [9]:
instance.activities[:4]

[Activity(modes=[Mode(duration=0, demands=[0, 0, 0, 0, 0], skill_requirements=None)], successors=[3, 2, 1, 8], delays=[0, 0, 0, 0], optional=False, selection_groups=[], name=''),
 Activity(modes=[Mode(duration=2, demands=[5, 7, 8, 4, 6], skill_requirements=None)], successors=[10], delays=[2], optional=False, selection_groups=[], name=''),
 Activity(modes=[Mode(duration=9, demands=[10, 8, 0, 8, 10], skill_requirements=None)], successors=[4, 11, 7], delays=[5, 9, 0], optional=False, selection_groups=[], name=''),
 Activity(modes=[Mode(duration=6, demands=[9, 9, 0, 4, 5], skill_requirements=None)], successors=[9], delays=[3], optional=False, selection_groups=[], name='')]

## MPLIB format

Multi-project RCPSP handles multiple projects with shared resources, downloaded from https://www.projectmanagement.ugent.be/research/project_scheduling/RCMPSP.

In [10]:
instance = parse("data/MPLIB1_Set1_0.rcmp", instance_format="mplib")

In [11]:
(instance.num_resources, instance.num_activities, instance.num_projects)

(4, 372, 6)

Each project contains a list of activity indices that belong to the project, and a common release date.

In [12]:
instance.projects[:2]

[Project(activities=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61], release_date=0),
 Project(activities=[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123], release_date=0)]

## RCPSP-PS and ASLIB format
In the RCPSP with flexible project structures (RCPSP-PS), there are choices about which activities to schedule.

1. Activities can be optional - they don't all need to be scheduled
2. Activities each have a set of associated selection groups
3. If an activity is scheduled, then for each selection group, at least one activity must also be scheduled

Our library support two common RCPSP-PS formats.

### RCPSP-PS

The "RCPSP-PS" format is the format used by [Van der Beek et al. (2024)](https://www.sciencedirect.com/science/article/pii/S0377221724008269).

In [13]:
instance = parse("data/rcpsp_ps.txt", instance_format="rcpsp_ps")

In [14]:
(instance.num_resources, instance.num_activities)

(4, 136)

In [15]:
activity = instance.activities[0]  # source
activity

Activity(modes=[Mode(duration=0, demands=[0, 0, 0, 0], skill_requirements=None)], successors=[1, 2], delays=None, optional=False, selection_groups=[[1, 2]], name='')

The source activity must always be present (`optional=False`).
It has one selection group consisting of activities 1 and 2.
This means that either activity 1 or activity 2 must be scheduled.

In [16]:
activity = instance.activities[1]
activity

Activity(modes=[Mode(duration=0, demands=[0, 0, 0, 0], skill_requirements=None)], successors=[3, 4], delays=None, optional=True, selection_groups=[[3], [4]], name='')

This activity has two selection groups, one with activity 3, and the other with activity 4.
If this activity is scheduled, then both activity 3 and activity must be scheduled.

### ASLIB
The "ASLIB" format is the format used for RCPSP with alternative subgraph (RCPSP-AS) instances at https://www.projectmanagement.ugent.be/research/project_scheduling/rcpspas.

Like the RCPSP-PS, the RCPSP-AS also has optional tasks and selection groups, but they use slightly different concepts (branches and subgraphs). 

**Important**: RCPSP-AS instances from the ASLIB instance set contain multiple file parts (a, b, c).
Our library parses _merged part (a) and (b)_ files, meaning that you have to manually concatenate files first.
See the instance file for details.

In [17]:
instance = parse("data/aslib0_0.rcp", instance_format="aslib")

In [18]:
(instance.num_resources, instance.num_activities)

(5, 122)

In [19]:
activity = instance.activities[0]  # source
activity

Activity(modes=[Mode(duration=0, demands=[0, 0, 0, 0, 0], skill_requirements=None)], successors=[1, 13, 25, 37, 49], delays=None, optional=False, selection_groups=[[1, 13, 25, 37, 49]], name='')

In [20]:
activity = instance.activities[1]
activity

Activity(modes=[Mode(duration=0, demands=[0, 0, 0, 0, 0], skill_requirements=None)], successors=[2, 3, 4, 5, 6, 7], delays=None, optional=True, selection_groups=[[2], [3], [4], [5], [6], [7]], name='')

### MSLIB

The "MSLIB" format is the format used for multi-skilled RCPSP (MSRCPSP) instances at https://www.projectmanagement.ugent.be/research/project_scheduling/MSRCPSP.

In [21]:
instance = parse("data/MSLIB_Set1_1.msrcp", instance_format="mslib")

The MSRCPSP extends the RCPSP by introducing _skills_. Each instance has a set of skills:

In [22]:
instance.skills

[0, 1, 2, 3]

Resources may or may not have the corresponding skill:

In [23]:
instance.resources[0]

Resource(capacity=1, renewable=True, skills=[True, True, True, True])

Resources all have a capacity of one, because each resource can be only used once for a given activity.

Activity modes now require a number of skills:

In [24]:
instance.activities[1].modes[0]

Mode(duration=7, demands=[0, 0, 0, 0], skill_requirements=[0, 3, 0, 0])

Note that the demands are all zero. Instead, demand is binary and determined by whether a resource or not.